In [15]:
import random
import numpy as np
from tqdm import tqdm
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import entropy, skew, kurtosis

In [17]:
def RandomPermutation():
    perm = list(range(8))
    random.shuffle(perm)
    return perm

def StupidPermutation():
    partialSums = [
    0,1,8,35,111,285,628,1230,2191,3606,5546,8039,11056,14506,
    18242,22078,25814,29264,32281,34774,36714,38129,39090,
    39692,40035,40209,40285,40312,40319,40320
    ]
    
    r = random.randint(0, partialSums[-1])
    numInv = 0
    while partialSums[numInv] < r:
        numInv += 1
    
    perm = list(range(8))
    for step in range(numInv):
        t1 = random.randint(0, 7)
        t2 = random.randint(0, 7)
        perm[t1], perm[t2] = perm[t2], perm[t1]
    return perm

In [19]:
def count_inv(arr):
    inv = 0
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return inv

def local_inversions(arr):
    return sum(arr[i] > arr[i+1] for i in range(7))

def fixed_points(arr):
    return sum(arr[i] == i for i in range(8))

def lis_length(arr):
    dp = [1]*8
    for i in range(8):
        for j in range(i):
            if arr[j] < arr[i]:
                dp[i] = max(dp[i], dp[j]+1)
    return max(dp)

def lds_length(arr):
    dp = [1]*8
    for i in range(8):
        for j in range(i):
            if arr[j] > arr[i]:
                dp[i] = max(dp[i], dp[j]+1)
    return max(dp)

In [21]:
def extract_features_of_set(permutations_1000):
    perms = np.array(permutations_1000)

    C = np.zeros((8,8))
    for p in perms:
        for pos,val in enumerate(p):
            C[pos,val] += 1

    C_norm = C / C.sum(axis=1, keepdims=True)
    ent = [entropy(C_norm[i] + 1e-12) for i in range(8)]
    uniform = np.ones(8)/8
    KL = []
    for i in range(8):
        row = C_norm[i] + 1e-12
        KL.append(np.sum(row * np.log(row / uniform)))

    invs = np.array([count_inv(p) for p in perms])
    locinv = np.array([local_inversions(p) for p in perms])
    fixp = np.array([fixed_points(p) for p in perms])
    lisv = np.array([lis_length(p) for p in perms])
    ldsv = np.array([lds_length(p) for p in perms])

    feats = []
    for arr in [invs, locinv, fixp, lisv, ldsv]:
        feats += [
            arr.mean(), arr.std(),
            np.quantile(arr, 0.1),
            np.quantile(arr, 0.5),
            np.quantile(arr, 0.9),
            skew(arr),
            kurtosis(arr)
        ]

    feats += ent
    feats += KL

    return np.array(feats, dtype=float)

In [23]:
def generate_dataset(num_sets):

    X = []
    y = []

    print("Генерация датасета...")
    for _ in tqdm(range(num_sets)):
        perms = [RandomPermutation() for _ in range(1000)]
        X.append(extract_features_of_set(perms))
        y.append(1)

    for _ in tqdm(range(num_sets)):
        perms = [StupidPermutation() for _ in range(1000)]
        X.append(extract_features_of_set(perms))
        y.append(0)

    return np.array(X), np.array(y)

In [39]:
X_train, y_train = generate_dataset(500)
X_test,  y_test  = generate_dataset(200)

print("Обучение модели...")
model = LGBMClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.05,
    min_child_samples=20
)
model.fit(X_train, y_train)

pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
print("Accuracy:", acc)

Генерация датасета...


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:25<00:00, 19.34it/s]


Генерация датасета...


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:10<00:00, 19.44it/s]

Обучение модели...
[LightGBM] [Info] Number of positive: 500, number of negative: 500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8799
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [41]:
feature_names = []

blocks = ["inv", "locinv", "fixp", "lis", "lds"]
stats = ["mean", "std", "q10", "median", "q90", "skew", "kurt"]

for blk in blocks:
    for s in stats:
        feature_names.append(f"{blk}_{s}")

for i in range(8):
    feature_names.append(f"entropy_pos_{i}")

for i in range(8):
    feature_names.append(f"KL_pos_{i}")

In [43]:
import pandas as pd
import matplotlib.pyplot as plt

feature_names = []

blocks = ["inv", "locinv", "fixp", "lis", "lds"]
stats = ["mean", "std", "q10", "median", "q90", "skew", "kurt"]

for blk in blocks:
    for s in stats:
        feature_names.append(f"{blk}_{s}")

for i in range(8):
    feature_names.append(f"entropy_pos_{i}")

for i in range(8):
    feature_names.append(f"KL_pos_{i}")

assert len(feature_names) == len(model.feature_importances_)

df_imp = pd.DataFrame({
    "feature": feature_names,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

print(df_imp)


          feature  importance
0        inv_mean         413
14      fixp_mean         200
1         inv_std          48
6        inv_kurt          44
5        inv_skew          19
13    locinv_kurt           5
28       lds_mean           1
8      locinv_std           1
12    locinv_skew           1
39  entropy_pos_4           0
31     lds_median           0
32        lds_q90           0
33       lds_skew           0
34       lds_kurt           0
35  entropy_pos_0           0
36  entropy_pos_1           0
37  entropy_pos_2           0
38  entropy_pos_3           0
49       KL_pos_6           0
40  entropy_pos_5           0
41  entropy_pos_6           0
48       KL_pos_5           0
30        lds_q10           0
43       KL_pos_0           0
44       KL_pos_1           0
45       KL_pos_2           0
46       KL_pos_3           0
47       KL_pos_4           0
42  entropy_pos_7           0
25        lis_q90           0
29        lds_std           0
16       fixp_q10           0
2         

In [45]:
DATA_PATH = 'data/209_permutations/permutations.in'
OUTPUT_PATH = 'data/209_permutations/output.txt'

def read_input_file(path=DATA_PATH):
    with open(path, "r") as f:
        lines = f.read().strip().split()

    n = int(lines[0])
    perms = []
    idx = 1

    for _ in range(n):
        block = []
        for _ in range(1000):
            p = list(map(int, lines[idx:idx+8]))
            idx += 8
            block.append(p)
        perms.append(block)

    return n, perms

def apply_model_to_file(model, input_path=DATA_PATH, output_path=OUTPUT_PATH):
    n, blocks = read_input_file(input_path)

    X = []
    print("Extracting features for", n, "blocks...")
    for block in blocks:
        X.append(extract_features_of_set(block))
    X = np.array(X)

    print("Predicting...")
    proba = model.predict_proba(X)[:, 1]

    order = np.argsort(-proba)

    print("Saving to", output_path)
    with open(output_path, "w") as f:
        for idx in order:
            f.write(str(idx) + "\n")

apply_model_to_file(model)


Extracting features for 200 blocks...
Predicting...
Saving to data/209_permutations/output.txt
